In [42]:
import json
import pandas as pd
from datetime import datetime, timezone

# obtains timestamp
def get_timestamp(filename):
  try:
    ts_str = filename.split('.')[0]
    ts_float = float(ts_str)
    return datetime.fromtimestamp(ts_float, tz=timezone.utc)
  except Exception:
    return None

# read JSON file
with open('data/segments_data_round3_day1_classification.json', 'r') as file:
  data = json.load(file)

# extract data
segments = []
drives = []
cameras = []
images = []
camera_images = []

cam_img_counter = 0
images_counter = 0

for segment_id, segment_data in data.items():
  segment_timestamps = []
  # traverse through hierarchy for timestamp
  for drive_id, drive_data in segment_data.items():
    dir_day = drive_data.get('dir_day')
    dir_pass = drive_data.get('dir_pass')
    drive_timestamps = []

    # only instances starting with cam (ignores dir_day and dir_pass)
    camera_keys = [k for k in drive_data.keys() if k.startswith('cam')]

    for cam_name in camera_keys:
      # obtains camera data (color and its files; depth and its files)
      cam_data = drive_data[cam_name]
      for img_type in ['color', 'depth']:
        # focuses on images by categories in camera data
        if img_type in cam_data:
          # processes filenames of current type (color or depth)
          for filename in cam_data[img_type]:
            # obtains timestamps
            ts = get_timestamp(filename)
            if ts:
              segment_timestamps.append(ts)
              drive_timestamps.append(ts)
            
            # adds images information
            images.append({
              'image_id': images_counter,
              'file_name': filename,
              'type': img_type
            })

            images_counter += 1

            # adds camera and images (linked) information
            camera_images.append({
              'id': cam_img_counter,
              'camera_id': f"{drive_id}_{cam_name}",
              'image_id': filename
            })

            cam_img_counter += 1

      # adds cameras information
      cameras.append({
        'camera_id': f"{drive_id}_{cam_name}",
        'drive_id': drive_id,
        'segment_id': segment_id,
        'name': cam_name
      })

  # get earliest timestamp of the drives for the segment 
  segment_time_recorded = min(segment_timestamps) if segment_timestamps else None

  # add segments information
  segments.append({
    'segment_id': segment_id,
    'location': 'Fort Wayne, IN',
    'date_recorded': segment_time_recorded.date()
  })

  # add drives information
  drives.append({
    'drive_id': drive_id,
    'segment_id': segment_id,
    'dir_day': dir_day,
    'dir_pass': dir_pass,
    'time_driven': drive_timestamps
  })


segment_df = pd.DataFrame(segments)
drives_df = pd.DataFrame(drives)
cameras_df = pd.DataFrame(cameras)
images_df = pd.DataFrame(images)
camera_images_df = pd.DataFrame(camera_images)

display(segment_df.head())
display(drives_df.head())
display(cameras_df.head())
display(images_df.head())
display(camera_images_df.head())



,segment_id,location,date_recorded
0,segment_7358,"Fort Wayne, IN",2025-03-09
1,segment_7352,"Fort Wayne, IN",2025-03-09
2,segment_7348,"Fort Wayne, IN",2025-03-09
3,segment_7360,"Fort Wayne, IN",2025-03-09
4,segment_26962,"Fort Wayne, IN",2025-03-09


,drive_id,segment_id,dir_day,dir_pass,time_driven
0,drive_0,segment_7358,E:\FW_Round3_Day1_3_9,pass0,"[2025-03-09 18:09:42+00:00, 2025-03-09 18:09:4..."
1,drive_0,segment_7352,E:\FW_Round3_Day1_3_9,pass0,"[2025-03-09 18:10:16+00:00, 2025-03-09 18:10:1..."
2,drive_0,segment_7348,E:\FW_Round3_Day1_3_9,pass0,"[2025-03-09 18:11:12+00:00, 2025-03-09 18:11:1..."
3,drive_0,segment_7360,E:\FW_Round3_Day1_3_9,pass0,"[2025-03-09 18:11:19+00:00, 2025-03-09 18:11:1..."
4,drive_0,segment_26962,E:\FW_Round3_Day1_3_9,pass0,"[2025-03-09 18:11:27+00:00, 2025-03-09 18:11:2..."


,camera_id,drive_id,segment_id,name
0,drive_0_cam1,drive_0,segment_7358,cam1
1,drive_0_cam2,drive_0,segment_7358,cam2
2,drive_0_cam1,drive_0,segment_7352,cam1
3,drive_0_cam2,drive_0,segment_7352,cam2
4,drive_0_cam1,drive_0,segment_7348,cam1


,image_id,file_name,type
0,0,1741543782.793046.jpg,color
1,1,1741543782.808574.jpg,color
2,2,1741543782.835759.jpg,color
3,3,1741543782.902475.jpg,color
4,4,1741543782.969288.jpg,color


,id,camera_id,image_id
0,0,drive_0_cam1,1741543782.793046.jpg
1,1,drive_0_cam1,1741543782.808574.jpg
2,2,drive_0_cam1,1741543782.835759.jpg
3,3,drive_0_cam1,1741543782.902475.jpg
4,4,drive_0_cam1,1741543782.969288.jpg
